# Imports

Environment Setup
1. create an isolated env 

conda create -n tinygesture python=3.10 -y 
conda activate tinygesture

2. PyTorch + CUDA 12.1 (change to 11.8 if your driver is older) 

conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

3. Jupyter + misc libs 

pip install notebook numpy scipy matplotlib pillow tqdm

4. pip install --upgrade pip wheel setuptools

5. PyTorch + CUDA
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [27]:
from pathlib import Path
from typing import Sequence, Tuple, List, Dict, Callable, Optional
from __future__ import annotations
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader,random_split
import argparse, textwrap
__all__ = [
    "RadarDataset",
    "build_dataloaders",
]
import platform, sys
print(torch.cuda.is_available(), torch.cuda.get_device_name(0))

True NVIDIA GeForce RTX 4060 Laptop GPU


TinyRadarNN gesture‑dataset loader (supports .txt, .npy, .png)

Directory layout expected
-------------------------
```
Datasets/
    5G \ data/
        FingerSlider/
            session_0/
                0_FingerSlider.txt
                1_FingerSlider.txt
                ...
            session_1/
        PalmTilt/
        ...
```

Each **clip** is stored in a text file where whitespace or comma separates floating‑point numbers laid out row‑wise (first row = sweep‑0, etc.).  The loader will infer the number of time‑steps *T* from the file length and an optional `range_bins` argument.

If you have pre‑converted `.npy` tensors or image‑encoded spectrograms (`.png`), set `file_ext="npy"` or `file_ext="png"` in the constructor; everything else stays the same.

# RadarDatset Loader

In [33]:
# helper
NUMERIC_START = tuple("0123456789-+.")


def _numeric_row(line: str) -> Optional[List[float]]:
    """Return list of floats if the line starts with a numeric token else None."""
    ls = line.strip()
    if not ls or ls[0] not in NUMERIC_START:
        return None
    try:
        return [float(tok) for tok in ls.split()]
    except ValueError:
        return None  # non‑numeric token – treat as header


def _load_clip_txt(path: Path, range_bins: int) -> torch.Tensor:
    """Load a .txt clip → (T, R) float32 tensor, ignore arbitrary headers."""
    rows: List[List[float]] = []
    with path.open("r") as fh:
        for ln in fh:
            row = _numeric_row(ln)
            if row is not None:
                rows.append(row)
    if not rows:
        #raise ValueError(f"{path}: empty or no numeric data found")
        return None

    mat = np.asarray(rows, dtype=np.float32)  # (T, R?)
    if mat.shape[1] != range_bins:
        raise ValueError(
            f"{path.name}: expected {range_bins} range‑bins, got {mat.shape[1]}"
        )
    return torch.from_numpy(mat)  # (T, R)


In [41]:
class RadarDataset(Dataset):
    """TinyRadarNN gesture dataset (txt / npy / png)."""

    def __init__(
        self,
        root: os.PathLike,
        gesture_names: Sequence[str],
        *,
        range_bins: int = 64,
        file_type: str | None = None,
    ) -> None:
        self.root = Path(root)
        self.gesture_names = list(gesture_names)
        self.g2idx = {g: i for i, g in enumerate(self.gesture_names)}
        self.range_bins = range_bins

        self.file_type = file_type  # "txt", "npy", "png" or None → auto
        self.index: List[Tuple[Path, int]] = []  # (path, label)
        self._build_index()

    # ------------------------------------------------------------------ index
    def _build_index(self) -> None:
        """Populate `self.index` with (Path, label) tuples, skipping bad clips."""
        exts = {".txt", ".npy", ".png"}

        for g in self.gesture_names:
            # Primary path: <root>/<gesture>/**            (5G layout)
            g_dir = self.root / g
            if g_dir.is_dir():
                candidates = g_dir.rglob("*")
            else:  # fallback deep search (11G layouts)
                candidates = self.root.rglob(f"**/{g}/**/*")

            for p in candidates:
                if not p.is_file() or p.suffix.lower() not in exts:
                    continue

                # Skip empty .txt clips right here to avoid runtime crashes
                if p.suffix.lower() == ".txt":
                    if _load_clip_txt(p, self.range_bins) is None:
                        continue  # no numeric data → ignore clip

                self.index.append((p, self.g2idx[g]))

        if not self.index:
            raise FileNotFoundError(f"No clips found under {self.root}")

        # Auto‑detect file type if user did not specify one
        self.file_type = self.file_type or self.index[0][0].suffix[1:]  # drop dot
    
    def __len__(self) -> int:
        return len(self.index)

    def __getitem__(self, i):
        path, label = self.index[i]
        if self.file_type == "txt":
            x = _load_clip_txt(path, self.range_bins)
        elif self.file_type == "npy":
            x = torch.from_numpy(np.load(path)).float()
        elif self.file_type == "png":
            import PIL.Image as Image
            x = torch.from_numpy(np.asarray(Image.open(path)).astype(np.float32))
        else:
            raise ValueError(f"Unsupported file_type {self.file_type}")
        return x, label

In [42]:
# conveniance funcstions
def build_dataloaders(
    root: os.PathLike,
    gesture_names: Sequence[str],
    *,
    batch_size: int = 16,
    range_bins: int = 64,
    split: float = 0.8,
    seed: int = 1,
) -> Tuple[DataLoader, DataLoader]:
    ds = RadarDataset(root, gesture_names, range_bins=range_bins)
    n_train = int(len(ds) * split)
    n_val = len(ds) - n_train
    g = torch.Generator().manual_seed(seed)
    train_ds, val_ds = random_split(ds, [n_train, n_val], generator=g)

    pin = torch.cuda.is_available()
    train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0, pin_memory=pin)
    val_dl   = DataLoader(val_ds,   batch_size, shuffle=False, num_workers=0, pin_memory=pin)
    return train_dl, val_dl


def inspect_dataset(ds: RadarDataset) -> None:
    table = []
    for g in ds.gesture_names:
        clips = [p for p, lbl in ds.index if lbl == ds.g2idx[g]]
        sample = ds[random.choice(range(len(clips)))]  # (x,label)
        table.append((g, len(clips), tuple(sample[0].shape), sample[0].min().item(), sample[0].max().item()))
    header = ("Gesture", "#clips", "shape(T,R)", "min", "max")
    colw = [max(len(str(x)) for x in col) for col in zip(header, *table)]
    def row(r):
        return "  ".join(str(x).ljust(w) for x, w in zip(r, colw))
    print(row(header))
    for r in table:
        print(row(r))

# Loading Data 5G

In [43]:
ROOT = Path(r"C:\Users\ioana\Documents\VS Projects\ML-on-Microcontrollers-Project-Ioana-Gidiuta\Datasets\5G\data")
GESTURES = ["FingerSlider","NoHand","PalmTilt","PullUp","PushDown","SwipeRL","xNoGesture"]

# 1) loaders
train_dl, val_dl = build_dataloaders(
    ROOT,
    gesture_names=GESTURES,
    batch_size=32,
    range_bins=64,        # stays 64 for your .txt format
)

# 2) quick sanity check on GPU

ap = argparse.ArgumentParser(description="TinyRadarNN dataset sanity‑check")
ap.add_argument("root", type=Path, help="Path to 5G or 11G data folder")
ap.add_argument("--range_bins", type=int, default=64, help="# range bins per sweep (for .txt)")
ap.add_argument("--gesture", action="append", required=True, help="Gesture names to include (repeatable)")
args = ap.parse_args()

ds = RadarDataset(args.root, args.gesture, range_bins=args.range_bins)
print(f"Loaded {len(ds)} clips across {len(ds.gesture_names)} classes: {ds.gesture_names}")
inspect_dataset(ds)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

FileNotFoundError: No clips found under C:\Users\ioana\Documents\VS Projects\ML-on-Microcontrollers-Project-Ioana-Gidiuta\Datasets\5G\data

In [ ]:
for x, y in train_dl:
    x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
    print( x.shape, "labels", y[:5].tolist())
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

# Train Test Split